In [1]:
import numpy as np
import pandas as pd
import fastai
from tqdm import tqdm_notebook as tqdm
from fastai.tabular import *

np.range = (lambda x:(x.min(), x.max()))

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv('test.csv')
structures = pd.read_csv('structures.csv')

In [3]:
train.isna().sum(), structures.isna().sum()

(id                          0
 molecule_name               0
 atom_index_0                0
 atom_index_1                0
 type                        0
 scalar_coupling_constant    0
 dtype: int64, molecule_name    0
 atom_index       0
 atom             0
 x                0
 y                0
 z                0
 dtype: int64)

## no nulls

In [4]:
train.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074


In [5]:
structures.head()

,molecule_name,atom_index,atom,x,y,z
0,dsgdb9nsd_000001,0,C,-0.012698,1.085804,0.008001
1,dsgdb9nsd_000001,1,H,0.002150,-0.006031,0.001976
2,dsgdb9nsd_000001,2,H,1.011731,1.463751,0.000277
3,dsgdb9nsd_000001,3,H,-0.540815,1.447527,-0.876644
4,dsgdb9nsd_000001,4,H,-0.523814,1.437933,0.906397


In [6]:
structures.describe()

,atom_index,x,y,z
count,2.358657e+06,2.358657e+06,2.358657e+06,2.358657e+06
mean,8.757349e+00,9.489178e-02,-3.337381e-01,6.241504e-02
std,5.592487e+00,1.655271e+00,1.989152e+00,1.445870e+00
min,0.000000e+00,-9.234889e+00,-9.933938e+00,-9.134765e+00
25%,4.000000e+00,-8.746097e-01,-1.826156e+00,-8.424896e-01
50%,9.000000e+00,5.183615e-02,-4.035932e-01,1.093207e-02
75%,1.300000e+01,1.116101e+00,1.373660e+00,9.394357e-01
max,2.800000e+01,9.382240e+00,1.018196e+01,7.894733e+00


In [7]:
train.describe()

,id,atom_index_0,atom_index_1,scalar_coupling_constant
count,4.658147e+06,4.658147e+06,4.658147e+06,4.658147e+06
mean,2.329073e+06,1.335689e+01,5.883966e+00,1.592165e+01
std,1.344691e+06,3.267712e+00,4.993943e+00,3.494198e+01
min,0.000000e+00,0.000000e+00,0.000000e+00,-3.621860e+01
25%,1.164536e+06,1.100000e+01,2.000000e+00,-2.549780e-01
50%,2.329073e+06,1.300000e+01,5.000000e+00,2.281130e+00
75%,3.493610e+06,1.600000e+01,8.000000e+00,7.390655e+00
max,4.658146e+06,2.800000e+01,2.800000e+01,2.048800e+02


In [8]:
set(train.molecule_name).intersection(set(test.molecule_name))

set()

In [21]:
tmp = train.merge(structures.rename(columns=lambda x: x+'_0'), left_on=['molecule_name', 'atom_index_0'], right_on=['molecule_name_0', 'atom_index_0'])
joined = tmp.merge(structures.rename(columns=lambda x: x+'_1'), left_on=['molecule_name', 'atom_index_1'], right_on=['molecule_name_1', 'atom_index_1'])

joined = joined.drop(columns=['molecule_name_0', 'molecule_name_1'])

In [22]:
(joined.atom_0.value_counts()/len(joined))*100

H    100.0
Name: atom_0, dtype: float64

In [23]:
(joined.atom_1.value_counts()/len(joined))*100

C    72.141755
H    20.794685
N     7.063560
Name: atom_1, dtype: float64

In [10]:
joined.shape

(4658147, 14)

In [11]:
np.range(joined.id)

(0, 4658146)

In [12]:
joined.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,H,0.002150,-0.006031,0.001976,C,-0.012698,1.085804,0.008001
1,4,dsgdb9nsd_000001,2,0,1JHC,84.8074,H,1.011731,1.463751,0.000277,C,-0.012698,1.085804,0.008001
2,7,dsgdb9nsd_000001,3,0,1JHC,84.8093,H,-0.540815,1.447527,-0.876644,C,-0.012698,1.085804,0.008001
3,9,dsgdb9nsd_000001,4,0,1JHC,84.8095,H,-0.523814,1.437933,0.906397,C,-0.012698,1.085804,0.008001
4,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,H,0.002150,-0.006031,0.001976,H,1.011731,1.463751,0.000277


In [13]:
GPD = joined.groupby(['atom_0', 'atom_1', 'type'])

GPD.agg(lambda x : len(x)/len(train))[['id']]

id
atom_0 atom_1 type          
H      C      1JHC  0.152296
              2JHC  0.244877
              3JHC  0.324245
       H      2JHH  0.081156
              3JHH  0.126791
       N      1JHN  0.009309
              2JHN  0.025601
              3JHN  0.035726

In [14]:
GPD.agg(lambda x : np.range(x))#[['scalar_coupling_constant']]

id                         molecule_name  \
atom_0 atom_1 type                                                        
H      C      1JHC   (0, 4658146)  (dsgdb9nsd_000001, dsgdb9nsd_133884)   
              2JHC  (20, 4658145)  (dsgdb9nsd_000007, dsgdb9nsd_133884)   
              3JHC  (58, 4658144)  (dsgdb9nsd_000009, dsgdb9nsd_133884)   
       H      2JHH   (1, 4658120)  (dsgdb9nsd_000001, dsgdb9nsd_133884)   
              3JHH  (23, 4658138)  (dsgdb9nsd_000007, dsgdb9nsd_133884)   
       N      1JHN  (10, 4656569)  (dsgdb9nsd_000002, dsgdb9nsd_133808)   
              2JHN  (18, 4658130)  (dsgdb9nsd_000005, dsgdb9nsd_133884)   
              3JHN  (73, 4658139)  (dsgdb9nsd_000010, dsgdb9nsd_133884)   

                   atom_index_0 atom_index_1        scalar_coupling_constant  \
atom_0 atom_1 type                                                             
H      C      1JHC      (1, 28)      (0, 18)               (66.6008, 204.88)   
              2JHC      (0, 28)      (0, 17)             (-36.2186, 42.8192)   
              3JHC      (1, 28)      (0, 18)             (-18.5821, 76.0437)   
       H      2JHH      (1, 27)      (2, 28)             (-35.1761, 11.8542)   
              3JHH      (0, 25)      (3, 28)             (-3.02046, 17.4841)   
       N      1JHN      (0, 23)      (0, 11)              (24.3222, 80.4187)   
              2JHN      (2, 25)      (0, 11)             (-2.62085, 17.7436)   
              3JHN      (1, 25)      (0, 11)  (-3.1724099999999997, 10.9712)   

                                                        x_0  \
atom_0 atom_1 type                                            
H      C      1JHC        (-9.234888942000001, 9.382240438)   
              2JHC        (-9.234888942000001, 9.382240438)   
              3JHC        (-9.234888942000001, 9.382240438)   
       H      2JHH        (-9.218969711, 8.220769838999999)   
              3JHH         (-7.933076547000001, 7.04495728)   
       N      1JHN  (-7.942035177999999, 6.608726142999999)   
              2JHN        (-7.933076547000001, 8.159108193)   
              3JHN        (-7.735907448, 8.072174217999999)   

                                                          y_0  \
atom_0 atom_1 type                                              
H      C      1JHC                (-9.494159758, 9.714469231)   
              2JHC                (-9.494159758, 9.714469231)   
              3JHC                (-9.494159758, 9.714469231)   
       H      2JHH                (-8.917475925, 8.118444459)   
              3JHH                (-8.112642726, 7.960800674)   
       N      1JHN                 (-8.038531287, 6.92555625)   
              2JHN          (-7.350395892000001, 8.050028205)   
              3JHN  (-7.3163323039999995, 7.6339779839999995)   

                                                   z_0  \
atom_0 atom_1 type                                       
H      C      1JHC         (-9.134764787, 7.637577918)   
              2JHC         (-9.134764787, 7.637577918)   
              3JHC         (-9.134764787, 7.637577918)   
       H      2JHH  (-9.134764787, 7.1846107329999995)   
              3JHH         (-9.134764787, 6.554751062)   
       N      1JHN    (-6.73856946, 6.775142917999999)   
              2JHN  (-6.750298797, 6.7791448260000005)   
              3JHN   (-6.716641838999999, 6.680448625)   

                                                        x_1  \
atom_0 atom_1 type                                            
H      C      1JHC              (-8.677875692, 8.420892703)   
              2JHC              (-7.317725985, 7.468888258)   
              3JHC        (-7.347374907000001, 8.420892703)   
       H      2JHH  (-9.234888942000001, 8.230467342999999)   
              3JHH  (-7.942035177999999, 8.230467342999999)   
       N      1JHN              (-7.019292425, 6.793022522)   
              2JHN              (-7.019292425, 6.793022522)   
              3JHN        (-7.311588747999999, 8.

In [15]:
tmp = test.merge(structures.rename(columns=lambda x: x+'_0'), left_on=['molecule_name', 'atom_index_0'], right_on=['molecule_name_0', 'atom_index_0'])
joined = tmp.merge(structures.rename(columns=lambda x: x+'_1'), left_on=['molecule_name', 'atom_index_1'], right_on=['molecule_name_1', 'atom_index_1'])
joined = joined.drop(columns=['molecule_name_0', 'molecule_name_1'])
GPD = joined.groupby(['atom_0', 'atom_1', 'type'])
GPD.agg(lambda x : len(x)/len(test))[['id']]

id
atom_0 atom_1 type          
H      C      1JHC  0.151907
              2JHC  0.244713
              3JHC  0.324081
       H      2JHH  0.081071
              3JHH  0.126693
       N      1JHN  0.009657
              2JHN  0.025713
              3JHN  0.036166

## train test distributions look similar and are disjoint on molecules

In [16]:
structures.head(n=10)

,molecule_name,atom_index,atom,x,y,z
0,dsgdb9nsd_000001,0,C,-0.012698,1.085804,0.008001
1,dsgdb9nsd_000001,1,H,0.002150,-0.006031,0.001976
2,dsgdb9nsd_000001,2,H,1.011731,1.463751,0.000277
3,dsgdb9nsd_000001,3,H,-0.540815,1.447527,-0.876644
4,dsgdb9nsd_000001,4,H,-0.523814,1.437933,0.906397
5,dsgdb9nsd_000002,0,N,-0.040426,1.024108,0.062564
6,dsgdb9nsd_000002,1,H,0.017257,0.012545,-0.027377
7,dsgdb9nsd_000002,2,H,0.915789,1.358745,-0.028758
8,dsgdb9nsd_000002,3,H,-0.520278,1.343532,-0.775543
9,dsgdb9nsd_000003,0,O,-0.034360,0.977540,0.007602


In [17]:
structures.describe()

,atom_index,x,y,z
count,2.358657e+06,2.358657e+06,2.358657e+06,2.358657e+06
mean,8.757349e+00,9.489178e-02,-3.337381e-01,6.241504e-02
std,5.592487e+00,1.655271e+00,1.989152e+00,1.445870e+00
min,0.000000e+00,-9.234889e+00,-9.933938e+00,-9.134765e+00
25%,4.000000e+00,-8.746097e-01,-1.826156e+00,-8.424896e-01
50%,9.000000e+00,5.183615e-02,-4.035932e-01,1.093207e-02
75%,1.300000e+01,1.116101e+00,1.373660e+00,9.394357e-01
max,2.800000e+01,9.382240e+00,1.018196e+01,7.894733e+00


In [20]:
(structures.atom.value_counts()/len(structures))*100

H    51.231993
C    35.262694
O     7.766581
N     5.611710
F     0.127021
Name: atom, dtype: float64

In [19]:
structures.atom.unique()

array(['C', 'H', 'N', 'O', 'F'], dtype=object)